In [27]:
import re
import json
import difflib

In [28]:
with open("intent_keywords.json") as f:
    intent_keywords = json.load(f)

with open("intent_patterns.json") as f:
    intent_patterns = json.load(f)

with open("intent_responses.json") as f:
    intent_responses = json.load(f)

print("[DEBUG] Loaded intents:", intent_keywords.keys())    
#print("[DEBUG] Loaded intents:", intent_keywords.values())   

[DEBUG] Loaded intents: dict_keys(['greeting', 'what_is_mf', 'types', 'nav_info', 'how_to_invest', 'minimum_investment_amount', 'sip', 'lumpsum', 'which_is_better_sip_lumpsum', 'tax_benefits', 'taxation', 'risk', 'redemption', 'exit_load', 'regulator_info', 'contact', 'farewell', 'creator_info', 'bye'])


In [29]:
def clean_text(user_input):
    text = user_input.lower()
    text = re.sub(r'[^\w\s]','',text)
    text = re.sub(r'\s+', ' ', text).strip()  
    return text

In [30]:
def match_keywords(user_input):
    user_input_clean = clean_text(user_input)
    user_words = re.findall(r'\b\w+\b', user_input_clean)

    for intent, keywords in intent_keywords.items():
        for keyword in keywords:
            keyword_clean = keyword.strip().lower()
            if keyword_clean in user_words:
                #response = intent_responses.get(intent,"I dont have response for that")
                print("if match_keywords intent: ", intent)
                return intent  # Return intent name, not response
    print("No match -->match_keywords ")
    return None

In [31]:
def fuzzy_match_keyword(user_input, threshold=0.7):
    # Flatten keyword list
    all_keywords = []
    for intent, kw_list in intent_keywords.items():
        all_keywords.extend(kw_list)

    matches = difflib.get_close_matches(user_input.lower(), all_keywords, n=1, cutoff=threshold)
    return matches[0] if matches else None

    # words = user_input.lower().split() # split entire sentence into words for matching
    
    # best_match = None
    # best_score = 0

    # for word in words:
    #     match = difflib.get_close_matches(word, all_keywords, n=1, cutoff=threshold)
    #     if match:
    #          # Get actual ratio score
    #         score = difflib.SequenceMatcher(None, word, match[0]).ratio()
    #         if score > best_score:
    #             best_score = score
    #             best_match = match[0]

    return best_match

In [32]:
def match_patterns(user_input):
    user_input_clean = clean_text(user_input)
    user_words = re.findall(r'\b\w+\b', user_input_clean)

    for intent, pattern_info in intent_patterns.items():
        required_words = pattern_info.get("required_words", [])
        one_word = pattern_info.get("one_word", False)
        
        # Check if required words match
        if one_word:
            for word in required_words:
                if word.lower() in user_words:
                    print("If cond match_pattern intent:", intent)
                    return intent        
        else:
            # all required words must be present (substring match allowed)
            if all(word.lower() in user_input_clean for word in required_words):
                print("else cond match_pattern intent:", intent)
                return intent
    
    print("no match--> match_pattern intent:", intent)            
    #return None



In [33]:
def get_response(user_input):

    # Step 1: check patterns 
    intent = match_patterns(user_input)
    print("step1: intent: ", intent)
   
        
    # Step 2: try keywords if no match patterns found
    if not intent:
        intent = match_keywords(user_input)

    # Step 2.1: If still no intent, try fuzzy matching
    if not intent:
        close_kw = fuzzy_match_keyword(user_input)
       
        if close_kw:
            # Find which intent this keyword belongs to
            for key_intent, kw_list in intent_keywords.items():
                if close_kw in kw_list:
                    intent = key_intent
                    print(f"Fuzzy matched {user_input} to keyword '{close_kw}' for intent '{intent}'")
                    break



    # Step 3: map intent if needed
    # If intent is in patterns and has a "map_to" field, use that
    # if intent is in patterns's required_words['nav'] then extract required words and then send to responses.json

    if intent and intent in intent_patterns:
        intent = intent_patterns[intent].get("map_to", intent)
        print("step3: intent get directory ", intent_patterns.get(intent,{}))  
        print("step3: required words for intent:", intent_patterns.get(intent, {}).get("required_words"))
        
        # extract required_words from intent 
        # ## this apply if intent text is differnt in response.json file and use required_word from patterns.json to match with reqponse.json
        ## not a suggested approach....but as per requirement did it.

        required_words = intent_patterns.get(intent, {}).get("required_words", [])
        if required_words:
            response_key = required_words[0]  # take first required word as response key
            intent_responses.get(response_key, "Sorry! I don't have a response for that.")
        else:
            print("No required words found for this intent.")

    #  Step 4:
    if intent:
        if intent in intent_responses:
            return intent_responses[intent]

In [34]:
while True:
    user_input= input("You: ")
   
    if user_input.lower() in ["exit" ,"quit"]:
        print("Bot: Goodbye!")
        break
    print("What user entered --> ", user_input)
    print("Bot: ",get_response(user_input))

What user entered -->  hi
If cond match_pattern intent: greeting
step1: intent:  greeting
step3: intent get directory  {'required_words': ['hello', 'hi', 'hey', 'good', 'morning', 'evening'], 'one_word': True}
step3: required words for intent: ['hello', 'hi', 'hey', 'good', 'morning', 'evening']
Bot:  Hello! I can help you with information about Mutual Funds in India.
What user entered -->  tell me mf
If cond match_pattern intent: what_is_mf
step1: intent:  what_is_mf
step3: intent get directory  {'required_words': ['mutual', 'fund', 'mf'], 'one_word': True}
step3: required words for intent: ['mutual', 'fund', 'mf']
Bot:  A mutual fund pools money from many investors to purchase a diversified portfolio of stocks, bonds, or other securities.
What user entered -->  thanks
If cond match_pattern intent: farewell
step1: intent:  farewell
step3: intent get directory  {'required_words': ['thank', 'thanks', 'thank you'], 'one_word': True}
step3: required words for intent: ['thank', 'thanks', '